In [1]:
# this note book extracts out the statistics from landsat fractional cover imagery
# it first generates a list of all avilable fc imagery and then matchs them to individual field sites based on a given
# day range around the field site measured date.

# the stats are then extracted out of the imagery within the day range given for each site to produce a single csv file 
# the outputs can then be used to assess the accuracy of the fractional cover data. 

In [2]:
import pandas as pd
import geopandas as gpd
import fnmatch
import os
import datetime
from datetime import timedelta
import glob
import pdb
import sys
import argparse
import csv
import subprocess
import rasterio
import numpy as np
import shutil

In [ ]:
# produce an updated list of available imagery form the shared satellite drive.

%run C:/Users/grants/code/file_man/list_of_files_multi_dir_fnmatch.py -d Z:/Landsat/wrs2/ -e *dilm[2-4]_zstdmask.img -o imglist_dil.csv

In [ ]:
df = pd.read_csv('imglist_dil.csv',header=None)
print(df.shape)
# get the path row and image date from the image file name
df['path_row'] = df[0].map(lambda x: str(x)[-35:-32]) + '_' + df[0].map(lambda x: str(x)[-31:-28])
df['img_date'] = df[0].map(lambda x: str(x)[-27:-23]) + '-' + df[0].map(lambda x: str(x)[-23:-21]) + '-' + df[0].map(lambda x: str(x)[-21:-19])
df['zone'] = df[0].map(lambda x: str(x)[-14:-13])
print (df['zone'].unique())
df.to_csv('imglist_dil_datetime_zone.csv')

In [3]:
import pandas as pd
import geopandas as gpd
import fnmatch
import os
import datetime
from datetime import timedelta
import glob
import pdb
import sys
import argparse
import csv
import subprocess
import rasterio
import numpy as np
import shutil



# make some temp dir's to put single site shp file and results into 

tempDir = './temp_individual_results'
tempshp = './temp_individual_shp'
    
# check if the temp dir exists and if it does remove it, otherwise create the new dir for the individual outputs
check_if_dir_exists = os.path.isdir(tempDir)
if check_if_dir_exists == True:
    shutil.rmtree(tempDir)
else:    
    os.makedirs(tempDir)
    
    
check_if_dir_exists = os.path.isdir(tempshp)
if check_if_dir_exists == True:
    shutil.rmtree(tempshp)
else:    
    os.makedirs(tempshp)

# read in the shape file and produce a date_time 
sd = gpd.read_file('C:/Users/grants/code/cal_val/field_shapefile/nt_rm_fieldSite_2021_wrs2sj_buff_test_double_wrs2.shp')
sd['date_time'] = pd.to_datetime(sd['Date'] ,yearfirst=False, dayfirst=True)
print (sd['date_time'])

# define the number of days either side of the field site measured date to extract stats from the available imagery 
number_of_days = 15

date_plus = number_of_days 
date_minus = -abs(number_of_days)
print (date_plus,date_minus)

# create the plus and minus date range 
sd["fwd_date"] = sd["date_time"] + timedelta(days=date_plus)
sd["bck_date"] = sd["date_time"] + timedelta(days=date_minus)


# generate a list of all the avilable imagery to extract the statstics from 





# iterrate over the sites in the shapefile and find the image matches and extract out the statistics 

for index, row in sd.iterrows():
    
    siteN = row['uid_2']
    
    sda = sd[(sd['uid_2'] == siteN )]
    print (sda['Site'])

    
    site = sda['Site']
    field_date = sda['date_time']
    print (field_date)
    # get the date range for the imagery stats requred
    search_date_plus =  row['fwd_date']
    search_date_minus = row['bck_date']                   
    print (search_date_plus) 
    print ('-------------------------------------------')
    # get the info to create the dir path to get a list of imagery 
    path_row = str(row['PATH']) + '_0' + str(row['ROW'])
    print (path_row)
    #search the img list and find matches
    df = pd.read_csv('imglist_dil_datetime_zone.csv',header=0)
    # make the img_date a time date column 
    df['img_dt'] = pd.to_datetime(df['img_date'], yearfirst=True, dayfirst=False) 
        
    dfs = df[(df['path_row'] == path_row)]
    print (dfs)
    imgS = dfs[dfs["img_dt"].isin(pd.date_range(search_date_minus,search_date_plus))]
    print (imgS)
    # get the zone from the imagery to inform the selection of the EPSG code to use.
    zone = imgS['zone'].unique()
    print (zone)
    
    # reproject the site polygon the same cordinate system of the imagery
    if zone == 2:
        
        sdsr = sda.to_crs("EPSG:32752")      
        print ('this is the projection ',sdsr.crs)
        print (list(sdsr))

    if zone == 3:
        sdsr = sda.to_crs("EPSG:32753")      
        print ('this is the projection ',sdsr.crs)
        print (sdsr)
        
    if zone == 4:
        sdsr.to_crs("EPSG:32754")      
        print ('this is the projection ',sdsr.crs)
        print (sdsr)
    
    temp_shp = sdsr[['uid', 'Station', 'Site', 'Date', 'C_Lat', 'C_Lon', 'NO_Lat', 'NO_Lon', 'ba_trees','ba_shrubs', 'ba_total', 'FPC', 'PPC', 'CC', 'PVg', 'NPVg', 'BGg', 'PV', 'NPV', 'BG','index_righ', 'AREA', 'PERIMETER', 'PR_', 'PR_ID', 'RINGS_OK', 'RINGS_NOK', 'PATH','ROW', 'MODE', 'SEQUENCE', 'WRSPR', 'PR', 'ACQDayL7', 'ACQDayL8', 'geometry']]
    
    
    
    temp_shp.to_file('./temp_individual_shp/temp_' + str(siteN) + '_.shp')
    
    shp_file = './temp_individual_shp/temp_' + str(siteN) + '_.shp'
    
    print ('Site name:',site)
    print ('field date:',field_date)
    print (path_row)
    print ('start search date:',search_date_minus) 
    for index, row in imgS.iterrows():
        print (row['0'])
    print ('end search date:',search_date_plus)

    #uid = uid
    for index, row in imgS.iterrows():
        
        site_name = str(site)
        
        date = str(field_date) 
        
        img = str(row['0'])
        print (img)
        
        nodata = 0 
        
        #uid = 'uid'
        
        shp = shp_file
        #print (shp)
        
        csv = './temp_individual_results/results_' + str(siteN) + '_' + str(index) + '.csv'
        
        #pdb.set_trace()
        
        print (csv)
        
        
        cmd = "python zonal_stats_single_cal_val_local.py --image %s --nodata %s --shape %s --csv %s"% (img,nodata,shp, csv) 
       
        os.system(cmd)
        #print (sdsr.crs)
        #print (sdsr)

        print ("...................")
        
   


    # select out the individual site and reproject it to projected coords UTM 52 or 53 and extract out the stats from 
    # the given img_selection list using the script - zonal_stats_single_perc.py
    
    # use geopandas to create the shp file in memory - may need to write it out as a file? if so make it temp and delete...
    
    # create a temp dir to save the individual results before concating them together 
  
   
       
# read in the individual band results and concatenate them to a single dataframe
tempRes = "./temp_individual_results/"

all_files = glob.glob(os.path.join(tempRes, "*.csv"))     # advisable to use os.path.join as this makes concatenation OS independent
    
df_from_each_file = (pd.read_csv(f) for f in all_files)
concatenated_df   = pd.concat(df_from_each_file,ignore_index=False, axis=0)

# export the results to a csv file
concatenated_df.to_csv('cal_val_dil_data_2021_results30days_test.csv')     


0   2021-09-04
1   2021-08-31
2   2021-09-04
3   2021-08-31
Name: date_time, dtype: datetime64[ns]
15 -15
0    BDA14A
2    BDA14A
Name: Site, dtype: object
0   2021-09-04
2   2021-09-04
Name: date_time, dtype: datetime64[ns]
2021-09-19 00:00:00
-------------------------------------------
106_073
       Unnamed: 0                                                  0 path_row  \
39102       39102  Z:/Landsat/wrs2/106_073\1988\198801\l5tmre_p10...  106_073   
39103       39103  Z:/Landsat/wrs2/106_073\1988\198801\l5tmre_p10...  106_073   
39104       39104  Z:/Landsat/wrs2/106_073\1988\198802\l5tmre_p10...  106_073   
39105       39105  Z:/Landsat/wrs2/106_073\1988\198802\l5tmre_p10...  106_073   
39106       39106  Z:/Landsat/wrs2/106_073\1988\198803\l5tmre_p10...  106_073   
...           ...                                                ...      ...   
40088       40088  Z:/Landsat/wrs2/106_073\2021\202108\l8olre_p10...  106_073   
40089       40089  Z:/Landsat/wrs2/106_073\2021\202109\

...................
Z:/Landsat/wrs2/106_072\2021\202109\l7tmre_p106r072_20210908_dilm2_zstdmask.img
./temp_individual_results/results_24_39098.csv
...................
Z:/Landsat/wrs2/106_072\2021\202109\l8olre_p106r072_20210916_dilm2_zstdmask.img
./temp_individual_results/results_24_39100.csv
...................
1    BDA15A
3    BDA15A
Name: Site, dtype: object
1   2021-08-31
3   2021-08-31
Name: date_time, dtype: datetime64[ns]
2021-09-15 00:00:00
-------------------------------------------
106_072
       Unnamed: 0                                                  0 path_row  \
38129       38129  Z:/Landsat/wrs2/106_072\1988\198801\l5tmre_p10...  106_072   
38130       38130  Z:/Landsat/wrs2/106_072\1988\198801\l5tmre_p10...  106_072   
38131       38131  Z:/Landsat/wrs2/106_072\1988\198802\l5tmre_p10...  106_072   
38132       38132  Z:/Landsat/wrs2/106_072\1988\198802\l5tmre_p10...  106_072   
38133       38133  Z:/Landsat/wrs2/106_072\1988\198803\l5tmre_p10...  106_072   
...      

In [ ]:
# read in the individual band results and concatenate them to a single dataframe
tempDir = "C:/Users/grants/code/cal_val/zonal_test/"

all_files = glob.glob(os.path.join(tempDir, "*.csv"))     # advisable to use os.path.join as this makes concatenation OS independent
    
df_from_each_file = (pd.read_csv(f) for f in all_files)
concatenated_df   = pd.concat(df_from_each_file,ignore_index=False, axis=0)

# export the results to a csv file
concatenated_df.to_csv('concatenate_results_test4.csv') 

In [ ]:
%run test.py

In [ ]:
sd

In [ ]:
sd['date_time'] = sd.obs_time.apply(pd.to_datetime)

In [ ]:
print (sd.WRSPR.unique())

In [ ]:
print (list(sd))

In [ ]:
sd["fwd_date"] = sd["date_time"] + timedelta(days=30)
sd["bck_date"] = sd["date_time"] + timedelta(days=-30)

In [ ]:
# generate the list of imagery based on parts of the file name 

# e.g. *dilm[2-3]_zstdmask.img 
# this finds all the dil imagery which has had the standard mask applied to it. 

In [ ]:
%run C:/Users/grants/code/file_man/list_of_files_multi_dir_fnmatch.py -d Z:/Landsat/wrs2/ -e *dilm[2-3]_zstdmask.img -o test_imglist_dev.csv

In [ ]:
df = pd.read_csv('test_imglist_dev.csv',header=None)
print(df.shape)
# get the path row and image date from the image file name
df['path_row'] = df[0].map(lambda x: str(x)[-35:-32]) + '_' + df[0].map(lambda x: str(x)[-31:-28])
df['img_date'] = df[0].map(lambda x: str(x)[-27:-23]) + '-' + df[0].map(lambda x: str(x)[-23:-21]) + '-' + df[0].map(lambda x: str(x)[-21:-19])
df['zone'] = df[0].map(lambda x: str(x)[-14:-13])
print (df['zone'].unique())
df.to_csv('imglist_proc.csv')


#for index, row in df.iterrows():
    #print (row[0],row['img_dt'])

In [ ]:
df = pd.read_csv('imglist_proc.csv',header=0)

print (list(df))

In [ ]:
sd = gpd.read_file('C:\\Users\\grants\\code\\cal_val\\nt_star_test_data.shp')
#sd['date_time'] = sd.obs_time.apply(pd.to_datetime)

print (list(sd))

for index, row in sd.iterrows():
    
    siteN = row['uid']
    
    sda = sd[(sd['uid'] == siteN )]
    print (sda['site'])
    zone = 2

    sds = gpd.GeoDataFrame(sda, geometry=sda['geometry'])
    sds.crs  
    
    
    if zone == 2:
        print ('this one')
        sdsr = sds.to_crs("EPSG:32752")      
        print (sdsr.crs)

    if zone == 3:
        
        sdsr = sds.to_crs("EPSG:32753")      
        sdsr.crs
        
        #sdsr.to_file('test7.shp', driver='ESRI Shapefile')  
    sdsr.crs

In [ ]:
%run C:/Users/grants/code/zonal/zonal_stats_single_perc.py -h

In [ ]:
ls

In [ ]:
search_date_plus = str(sda['fwd_date'].values)[2:12]
search_date_minus = str(sda['bck_date'].values)[2:12]


print ("---------------")
print (search_date_plus)
print (search_date_minus)
print ("---------------")

In [ ]:
sda['ROW'].values

In [ ]:
# get the info to create the dir path to get a list of imagery 
path_row = str(sda['PATH'][0]) + '_0' + str(sda['ROW'][0])
    
print ("the path: ", str(sda['PATH'][0])) 
print (str(sda['ROW'][0])) 
print ("''''''''''''''''''''''''''''''")
print ('this should be the path: ', path_row)
print ("''''''''''''''''''''''''''''''")
year = str(sda['obs_time'][:4])
year

In [ ]:
# read in the shape file and produce a date_time 
sd = gpd.read_file('C:\\Users\\grants\\code\\cal_val\\nt_star_test_data.shp')
sd['date_time'] = sd.obs_time.apply(pd.to_datetime)

number_of_days = 20

# define the number of days either side of the field site measured date to extract stats from the available imagery 
date_plus = number_of_days 
date_minus = -abs(number_of_days)

print (date_plus,date_minus)
# create the plus and minus date range 
sd["fwd_date"] = sd["date_time"] + timedelta(days=date_plus)
sd["bck_date"] = sd["date_time"] + timedelta(days=date_minus)


# iterrate over the sites in the shapefile and find the image matches and extract out the statistics 

for index, row in sd.iterrows():

    site_zonal = row
    #site_zonal.to_file("temp_shp.shp")
    
    #print (site_zonal.astype)
    site = row['site']
    field_date = row['date_time']
    
    # get the date range for the imagery stats requred
    search_date_plus = row['fwd_date']
    search_date_minus = row['bck_date']
    print ("---------------")
    print (search_date_plus)
    print (search_date_minus)
    print ("---------------")
    
    
    # get the info to create the dir path to get a list of imagery 
    path_row = str(row['PATH']) + '_0' + str(row['ROW'])
    print (path_row)
    year = str(row['obs_time'][:4])
    
    #search the img list and find matches
    df = pd.read_csv('imglist_proc.csv',header=0)
    # make the img_date a time date column 
    df['img_dt'] = df.img_date.apply(pd.to_datetime)
        
    dfs = df[(df['path_row'] == path_row)]
    
    
    
    imgS = dfs[dfs["img_dt"].isin(pd.date_range(search_date_minus,search_date_plus))]
    
    zone = imgS['zone'].unique()
    print (zone)
    #print (imgS)
    #if zone == 2:
        #site_zonal.crs 
    
    #print (img_selection.dtypes)
        
    #for row in imgS:
        
    print ('Site name:',site)
    print ('field date:',field_date)
    print (path_row)
    print ('start search date:',search_date_minus)    
    
    print (imgS[['img_date','0','zone']])    
    print ('end search date:',search_date_plus)
    
    
    print ("...................")
    
    site_zonal


    # select out the individual site and reproject it to projected coords UTM 52 or 53 and extract out the stats from 
    # the given img_selection list using the script - zonal_stats_single_perc.py
    
    # use geopandas to create the shp file in memory - may need to write it out as a file? if so make it temp and delete...
    
    # create a temp dir to save the individual results before concating them together 
    
   
       
    


In [ ]:
df['geometry'] = [Point(xy) for xy in zip(df['Long'], df['Lat'])]
# create a df with all of the attributes 
df2 = df[['geometry','Collectors','Date','Lat', 'Long','photo','Primary','Secondary','Tertiary','lu_code','lu_coden','Commodity','Variety','Management','Confidence','Comments','UIDkey']]
# create the geo data frame        
df2 = geopandas.GeoDataFrame(df2, geometry='geometry')
# set the projection and datum
df2.crs = "+init=epsg:4326"
# save out the geo data frame in esri shapefile format.
df2.to_file(shapeName, driver='ESRI Shapefile')  

In [ ]:
sdsr.crs 

In [ ]:
    if epsg_code == 32752:
        epsg_code_proj4 = '+proj=utm +zone=52 +south +ellps=WGS84 +datum=WGS84 +units=m +no_defs'
    
    if epsg_code == 32753:
        epsg_code_proj4 = '+proj=utm +zone=53 +south +ellps=WGS84 +datum=WGS84 +units=m +no_defs'

In [ ]:
ax = sdsr.plot()

In [ ]:
%run C:\Users\gwsta\code\zonal\zonal_stats_single_perc.py -h

In [ ]:
http://localhost:8888/edit/code/zonal/zonal_stats_single_perc.py

In [ ]:
    df = pd.read_csv(csvName, header=0)
    
    df['geometry'] = [Point(xy) for xy in zip(df['Long'], df['Lat'])]
    # create a df with all of the attributes 
    df2 = df[['geometry','Collectors','Date','Lat', 'Long','photo','Primary','Secondary','Tertiary','lu_code','lu_coden','Commodity','Variety','Management','Confidence','Comments','UIDkey']]
    # create the geo data frame        
    df2 = geopandas.GeoDataFrame(df2, geometry='geometry')
    # set the projection and datum
    df2.crs = "+init=epsg:4326"
    # save out the geo data frame in esri shapefile format.
    df2.to_file(shapeName, driver='ESRI Shapefile') 

In [ ]:
site_list=[]
site_date=[]


#for pathrow in sd.WRSPR.unique():
    
for index, row in sd.iterrows():
    
    site = row['site']
    field_date = row['date_time']
    
    # get the date range for the imagery stats requred
    search_date_plus = row['fwd_date']
    search_date_minus = row['bck_date']
    
    # get the info to create the dir path to get a list of imagery 
    path_row = str(row['PATH']) + '_0' + str(row['ROW'])
    year = str(row['obs_time'][:4])
    
    #month = row['obs_time'][5:7]
    
    print (path_row)
    
    print (search_date_minus)
    
    print (field_date)
    
    print (year)
        
    print (search_date_plus)
    
    print ("...................")
    
    dirname = 'C:/data/landsat/' + path_row + '/' + year + '/'
    
    print (dirname)
    print ("...................")
    
    pattern = '_zstdmask.img'
       
    
    img_list = []
    
    
    for root, dirs, files in os.walk(dirname):
        for file in files:
            if file.endswith(pattern):
                img = (os.path.join(root, file))
                img_list.append(img)
                #print (img)
    
    print (img_list)
    
    df = pd.DataFrame(img_list)
    
    

    #for img in img_list:
        
        #date = img[-27:-19]
        
        #if date > search_date_mimus & < search_date_minus: 
             
          

    

In [ ]:
dirname = 'D:/landsat/105_070/2011/'
    
print (dirname)
print ("...................")
    
pattern = '_zstdmask.img'
       
    
list_img = []
    
    
for root, dirs, files in os.walk(dirname):
    for file in files:
        if file.endswith(pattern):
            img = (os.path.join(root, file))
            list_img.append(img)
            print (img)
    
print (imglist)

In [ ]:
site_list=[]
site_date=[]


#for pathrow in sd.WRSPR.unique():
    
for index, row in sd.iterrows():
    
    site = row['site']
    date = row['date_time']
    year = row['obs_time'][:4]
    month = row['obs_time'][5:7]
    
    month_m1 = int(month)-1
    month_p1 = int(month)+1
    
    print (month_m1)
    print (month_p1)
    
    if month_p1 <=9:
        
        month_p1 = str(month_p1)[0:3] + '0' + str(month_p1)
        print ("...................")
        print (month_p1)
        print ("...................")
    if month_m1 <=9:   
        month_m1 = str(month_m1)[0:3] + '0' + str(month_m1)
        
    yearmonth = str(row['obs_time'][:4])+str(row['obs_time'][5:7])
    
    yearmonth_p1 = str(row['obs_time'][:4]+str(month_p1))
    yearmonth_m1 = str(row['obs_time'][:4]+str(month_m1))
    
    path_row = str(row['PATH']) + '_0' + str(row['ROW'])
    
    #print (site, ' ', date, ' ', year, ' ',month, ' ', path_row)
    print (site, ' ', date) 
    print ('plus month ', yearmonth_p1)
    print ('minus month ', yearmonth_m1)
    # select the folders to create the image list for each site
    
    # construct the file path from the site information 
    seach_month = [yearmonth_m1,yearmonth,yearmonth_p1]
    
    for month in seach_month:
        
            
        dirname = 'Z:/Landsat/wrs2/' + path_row + '/' + year + '/' + month + '/'
    
        print (dirname)
    
        pattern = '*_zstdmask.img'
    
        #imglist = fnmatch.filter(os.listdir(dirname), pattern)
    
        #print (imglist)
    
    